In [1]:
import sys
import ff

# sys.path.insert(0, "/Users/raymondiacobacci/Downloads/rcwaControl/")
# import rcwa  # type: ignore

import matplotlib.pyplot as plt
import numpy as np

import pandas as pd

h = 6.626070e-34  # Js Planck's constant
c = 2.997925e8  # m/s speed of light
k_B = 1.380649e-23  # J/K Boltzmann constant
q = 1.602176e-19  # C elementary charge
l_i = np.arange(350, 3001, 1)  # nm wavelength range
T_e = 2073.15  # K emitter temperature

blackbody_radiation = (
    lambda lambda_i, T: (2 * h * c**2)
    / ((np.exp((h * c) / (k_B * T * lambda_i * 1e-9)) - 1) * lambda_i**5)
    * 1e32
)

B_i = blackbody_radiation(l_i, T_e)  # 2073.15K blackbody

with open("homogeneous-film-emissivity.txt", "r") as f:
    p = f.readlines()
efficiency_array = ff.parse_data(p)

blackbody_transmission_array = [i * B_i for i in efficiency_array]

n_all = pd.read_excel("/Users/raymondiacobacci/LeiteLab-6_8/n-allHTMats-2.xlsx")
k_all = pd.read_excel("/Users/raymondiacobacci/LeiteLab-6_8/k-allHTMats.xlsx")

nb_B = (
    lambda lambda_i, T: (2 * c)
    / ((np.exp((h * c) / (k_B * T * lambda_i * 1e-9)) - 1) * lambda_i**4)
    * 1e23
)
nb_B_e = nb_B(l_i, T_e)
T_PV = 300
nb_B_PV = nb_B(l_i, T_PV)
nk_AlN = n_all["AlN"] + 1j * k_all["AlN"]
nk_W = n_all["W"] + 1j * k_all["W"]

# Testing of Kim's TORCWA

In [ ]:
'''
This tests the performance of the two situations
'''
emissivity_at_wavelength_declan = np.load("control_density.npy")

def plot_density(array):
    print(emissivity_at_wavelength_declan)
    print(array * B_i)
    print(emissivity_at_wavelength_declan - array * B_i)
    
    plt.plot(l_i, array * B_i, label="+ Rand Grating")
    plt.plot(l_i, emissivity_at_wavelength_declan, label="Control (Declan)")
    plt.plot(l_i, B_i, label="Blackbody radiation")
    plt.plot(l_i, emissivity_at_wavelength_declan - array * B_i, label="Difference in values")
    plt.axvline(
        x=np.argmax(B_i) + l_i[0], color="r", linestyle="--", label="Peak intensity"
    )
    plt.legend(loc="upper right")
    plt.ylabel("Power Density (W/cm^2/nm)")
    plt.xlabel("Wavelength (nm)")
    plt.title("Spectral Density")
    plt.show()

er = np.ones(shape = (100,))
er[50:60] = 0
ur = np.ones(shape=er.shape)
grating_depth = .473
grating_thickness = 1
latticeVectors = [grating_thickness, 1]

emissivity_at_wavelength = []

air_layer = rcwa.Layer(n = 1, thickness = 9999)
for wavelength_index in range(2651): # We have 2651 points
    wavelength_index = 1150
    AlN_grating = rcwa.Crystal(latticeVectors, er=(nk_AlN[wavelength_index]**2-1) * er + 1, ur=ur)
    AlN_layer = rcwa.Layer(crystal = AlN_grating, thickness = grating_depth)
    W_layer = rcwa.Layer(n=nk_W[wavelength_index], thickness=9998)
    stack = rcwa.LayerStack(
        AlN_layer, incident_layer=air_layer, transmission_layer=W_layer
    )
    source = rcwa.Source(wavelength=l_i[wavelength_index] / 1000.0)
    solver = rcwa.Solver(stack, source, 3)
    results = solver.solve()
    emissivity_at_wavelength.append(1 - results['RTot'])
    break

print(emissivity_at_wavelength)
# plot_density(emissivity_at_wavelength)

This is the S-matrix for the layer with thickness 0.473:
[[[[-0.42152638+0.24971995j  0.11078279+0.01290074j
     0.09501243+0.00639252j  0.        +0.j
     0.        +0.j          0.        +0.j        ]
   [ 0.11078279+0.01290074j -0.41884169+0.24536083j
     0.11078279+0.01290074j  0.        +0.j
     0.        +0.j          0.        +0.j        ]
   [ 0.09501243+0.00639252j  0.11078279+0.01290074j
    -0.42152638+0.24971995j  0.        +0.j
     0.        +0.j          0.        +0.j        ]
   [ 0.        +0.j          0.        +0.j
     0.        +0.j         -0.42152638+0.24971995j
     0.11078279+0.01290074j  0.09501243+0.00639252j]
   [ 0.        +0.j          0.        +0.j
     0.        +0.j          0.11078279+0.01290074j
    -0.41884169+0.24536083j  0.11078279+0.01290074j]
   [ 0.        +0.j          0.        +0.j
     0.        +0.j          0.09501243+0.00639252j
     0.11078279+0.01290074j -0.42152638+0.24971995j]]

  [[-0.47079776-0.68975461j -0.14257361+0.05727

# Testing of Raymond's rcwa-ctrl

In [3]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, "/Users/raymondiacobacci/ll/")
import solver_v2 as rc  # type: ignore


er = np.array([x%2 for x in range(100)])
er = np.ones(shape = er.shape)
ur = np.ones(shape = er.shape)

test_wavelength_index = 1150

n_harmonics = 0

air_layer = rc.Layer_(permittivity = er*2.6, permeability = ur, thickness = 99, n_harmonics = n_harmonics)
aln_grating = rc.Layer_(
    permittivity=nk_AlN[test_wavelength_index] ** 2 * er,
    permeability=ur,
    thickness=5,
    n_harmonics=n_harmonics,
)

my_solver = rc.Solver_(layer_stack = [air_layer, air_layer, air_layer], grating_period = 1, wavelength = 1, n_harmonics = n_harmonics)

grating_scattering_matrix = my_solver.internal_layer_scattering_matrices[0]

# print(grating_scattering_matrix.shape)
# print(f"Grating scattering matrix forward transmittance:\n{grating_scattering_matrix[:5,:5]}")
# print(my_solver.incident_scattering_matrix[:5,:5])

print(f'This is the global scattering matrix!\n{my_solver.global_scattering_matrix}')

# print(f'Emissivity: {my_solver.global_scattering_matrix[2*n_harmonics + 1:, :2*n_harmonics + 1]}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Incident Kz in layer 99:
[[1.61245155+0.j]]
Kz in layer 99
[[0.89790076+0.89790076j 0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j]]
This is the h_lambda matrix for a layer of thickness: 99
[[ 0.        +0.j         -0.70710678-0.70710678j]
 [ 0.89790076+0.89790076j  0.        +0.j        ]]
This is the s00 matrix for a layer of thickness: 99
[[-3.95523742e-32-5.63518214e-33j  5.59437097e-01+1.11022302e-16j]
 [ 4.40562903e-01-5.55111512e-17j  9.34195782e-32-1.41393689e-31j]]
This is the s01 matrix for a layer of thickness: 99
[[-5.59437097e-01-1.11022302e-16j  3.80373396e-32+8.45609991e-32j]
 [ 2.99548257e-32+6.65927222e-32j -4.40562903e-01+5.55111512e-17j]]
This is the s10 matrix for a layer of thickness: 99
[[-5.59437097e-01-5.55111512e-17j  3.80373396e-32+8.45609991e-32j]
 [ 2.99548257e-32+6.65927222e-32j -4.40562903e-01+1.11022302e-16j]]
This is the s11 matrix for a laye